In [ ]:
import ee
import urllib.request
import pandas as pd
import numpy as np

ee.Authenticate()
ee.Initialize()

In [ ]:
locations = pd.read_csv('../csv_folder/total_centrepoints.csv')
locations

## Copernicus Satellite images

### Single Image

In [ ]:
vis_min = 0  # Visualization settings for the image
vis_max = 10_000 # Visualization settings for the image
vis_bands = ['B4', 'B3', 'B2'] # Includes the bands for RGB 
imageDimensions = '512x512' # Set image size: '512x512' for CNN model

id_ = 1

# Centre point for satellite image
longitude = -0.125782
latitude = 51.508272

center = ee.Geometry.Point(longitude,latitude)

# Import Sentinel 2 data
s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
  .filterBounds(center)
  .sort('CLOUDY_PIXEL_PERCENTAGE',True)
  .filterDate('2021-06-01', '2021-07-30')
  .first()
)

# try:
global sentinel_footprint
exportArea = (ee.Geometry.BBox(west=longitude - 0.2,
                               south=latitude - 0.2,
                               east=longitude + 0.2,
                               north=latitude + 0.2))
# except:
#     print("No Footprint found")
#     continue


# This part is nicer in the JavaScript version, but I did not get it running for Python yet

s2Vis = {
    'region': exportArea,
    'dimensions': imageDimensions,
    'format': 'png',
    'min': vis_min,
    'max': vis_max,
    'bands': vis_bands,
    'gamma':1.5
}

s2_url = (s2.getThumbURL(s2Vis))


urllib.request.urlretrieve(s2_url, f"../raw_data/zoomed_photos/image{14509}_{round(longitude,2)}_{round(latitude,2)}.png")
print(s2_url)

### Looping Across DataFrame

In [ ]:
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 10_000 #Visualization settings for the thumbnail
vis_bands = ['B4', 'B3', 'B2'] #Includes the bands for RGB 
imageDimensions = '512x512' #Set thumbnail image size (can't be too big, or you run into problems)
start_location = 14508 # where you will start your loop based on the dataframe
end_location = 14512 # where you will end your loop based on the dataframe

for index,row in locations.iloc[start_location:end_location].iterrows():
    id_ = index
    longitude = row['longitude']
    latitude = row['latitude']

    center = ee.Geometry.Point(longitude,latitude)

    # Import Sentinel 2 data
    s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
      .filterBounds(center)
      .sort('CLOUDY_PIXEL_PERCENTAGE',True)
      .filterDate('2021-06-01', '202-07-30')
      .first()
    )

    try:
        global sentinel_footprint
        exportArea = (ee.Geometry.BBox(west=longitude - 0.02135,
                                       south=latitude - 0.02135,
                                       east=longitude + 0.02135,
                                       north=latitude + 0.02135))
    except:
        print("No Footprint found")
        continue


    # This part is nicer in the JavaScript version, but I did not get it running for Python yet

    s2Vis = {
        'region': exportArea,
        'dimensions': imageDimensions,
        'format': 'png',
        'min': vis_min,
        'max': vis_max,
        'bands': vis_bands,
        'gamma':1.5
    }

    s2_url = (s2.getThumbURL(s2Vis))

    try:
        urllib.request.urlretrieve(s2_url, f"../raw_data/zoomed_photos/image{id_}_{round(longitude,2)}_{round(latitude,2)}.png")
        print(f'finished number {id_} at {round(longitude,2)} : {round(latitude,2)}')
    except:
        print(f"No link for image {id_} found")
        continue
        
    if id_ == end_location:
        print('Congrats 🎊🎉🥳')

## ESA Worldcover Downloader

In [ ]:
from pystac_client import Client

import os
from rio_tiler.io import STACReader

from utils.labels import label_to_rgb

import matplotlib.pyplot as plt

### Single image

In [ ]:
stac_endopoint = 'https://services.terrascope.be/stac/'

collection_id = 'urn:eop:VITO:ESA_WorldCover_10m_2021_AWS_V2'

In [ ]:
longitude_world = -104.8

latitude_world = 38.8

bbox = [longitude_world - 0.02135, latitude_world - 0.02135, longitude_world + 0.02135, latitude_world + 0.02135]

client = Client.open(stac_endopoint)

search_results = client.search(
    collections=collection_id,
    bbox=bbox
)

item = search_results.get_all_items()[0]

# this is to tell GDAL we do not need authentication
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

# name of the map asset
asset = 'ESA_WORLDCOVER_10M_MAP'

with STACReader(None, item=item) as stac:
    img = stac.part(bbox, assets=asset)
    
land_use_array = label_to_rgb(img.data[0])


plt.imshow(land_use_array)

### Looping Across DataFrame

In [ ]:
stac_endopoint = 'https://services.terrascope.be/stac/'

collection_id = 'urn:eop:VITO:ESA_WorldCover_10m_2021_AWS_V2'

In [ ]:
start_location = 14509 # where you will start your loop based on the dataframe
end_location = 14510  # where you will end your loop based on the dataframe
target_data = []

for index,row in locations.iloc[start_location:end_location].iterrows():
        
    id_ = index
    longitude = row['longitude']
    latitude = row['latitude']
    
    bbox = [longitude - 0.02135, latitude - 0.02135, longitude + 0.02135, latitude + 0.02135]

    client = Client.open(stac_endopoint)

    search_results = client.search(
        collections=collection_id,
        bbox=bbox
    )

    item = search_results.get_all_items()[0]

    # this is to tell GDAL we do not need authentication
    os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

    # name of the map asset
    asset = 'ESA_WORLDCOVER_10M_MAP'

    with STACReader(None, item=item) as stac:
        img = stac.part(bbox, assets=asset)

    land_use_array = label_to_rgb(img.data[0])
    
    target_data.append(land_use_array)
    print(f'at {start_location + len(target_data)} of the arrays')
    
    if len(target_data)%250 == 0:
        target_array = np.array(target_data)
        np.save(f'../raw_data/ESA_worldcover/land_use_data_from_{start_location + len(target_data) - 250}_to_{start_location + len(target_data) - 1}.npy',target_array)
        
        target_data = []
        start_location += 250
        
        continue
    
    
    if start_location + len(target_data) == end_location:
        target_array = np.array(target_data)
        np.save(f'../raw_data/ESA_worldcover/land_use_data_from_{start_location + len(target_data) - 250}_to_{start_location + len(target_data) - 1}.npy',target_array)
        print('Congrats 🎊🎉🥳')
    
    else:
        continue